# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 14 2023 12:14PM,257353,10,SOIVA0000584,Ivaoes Animal Health,Released
1,Feb 14 2023 12:12PM,257352,10,SOTAH0000909,"Nextsource Biotechnology, LLC",Released
2,Feb 14 2023 12:12PM,257350,10,0086381431,ISDIN Corporation,Released
3,Feb 14 2023 12:12PM,257350,10,0086381432,ISDIN Corporation,Released
4,Feb 14 2023 12:12PM,257350,10,0086381436,ISDIN Corporation,Released
5,Feb 14 2023 12:12PM,257350,10,0086381437,ISDIN Corporation,Released
6,Feb 14 2023 12:12PM,257350,10,0086381434,ISDIN Corporation,Released
7,Feb 14 2023 12:12PM,257350,10,0086381433,ISDIN Corporation,Released
8,Feb 14 2023 12:12PM,257350,10,0086381025,ISDIN Corporation,Released
9,Feb 14 2023 12:12PM,257350,10,0086381033,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
44,257349,Released,3
45,257350,Released,49
46,257351,Released,1
47,257352,Released,1
48,257353,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257349,NaN,NaN,3.0
257350,NaN,NaN,49.0
257351,NaN,NaN,1.0
257352,NaN,NaN,1.0
257353,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257265,0.0,8.0,1.0
257266,0.0,1.0,0.0
257267,0.0,43.0,2.0
257274,0.0,3.0,2.0
257277,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257265,0,8,1
257266,0,1,0
257267,0,43,2
257274,0,3,2
257277,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257265,0,8,1
1,257266,0,1,0
2,257267,0,43,2
3,257274,0,3,2
4,257277,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257265,,8,1
1,257266,,1,
2,257267,,43,2
3,257274,,3,2
4,257277,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 14 2023 12:14PM,257353,10,Ivaoes Animal Health
1,Feb 14 2023 12:12PM,257352,10,"Nextsource Biotechnology, LLC"
2,Feb 14 2023 12:12PM,257350,10,ISDIN Corporation
51,Feb 14 2023 12:08PM,257351,10,"Snap Medical Industries, LLC"
52,Feb 14 2023 12:08PM,257349,10,ISDIN Corporation
55,Feb 14 2023 12:04PM,257348,20,"Exact-Rx, Inc."
61,Feb 14 2023 11:57AM,257347,10,"Citieffe, Inc."
62,Feb 14 2023 11:48AM,257345,20,Else Nutrition
63,Feb 14 2023 11:47AM,257346,10,MedStone Pharma LLC
70,Feb 14 2023 11:18AM,257343,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 14 2023 12:14PM,257353,10,Ivaoes Animal Health,,,1
1,Feb 14 2023 12:12PM,257352,10,"Nextsource Biotechnology, LLC",,,1
2,Feb 14 2023 12:12PM,257350,10,ISDIN Corporation,,,49
3,Feb 14 2023 12:08PM,257351,10,"Snap Medical Industries, LLC",,,1
4,Feb 14 2023 12:08PM,257349,10,ISDIN Corporation,,,3
5,Feb 14 2023 12:04PM,257348,20,"Exact-Rx, Inc.",,,6
6,Feb 14 2023 11:57AM,257347,10,"Citieffe, Inc.",,,1
7,Feb 14 2023 11:48AM,257345,20,Else Nutrition,,,1
8,Feb 14 2023 11:47AM,257346,10,MedStone Pharma LLC,,,7
9,Feb 14 2023 11:18AM,257343,10,"Methapharm, Inc.",,,21


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 14 2023 12:14PM,257353,10,Ivaoes Animal Health,1,,
1,Feb 14 2023 12:12PM,257352,10,"Nextsource Biotechnology, LLC",1,,
2,Feb 14 2023 12:12PM,257350,10,ISDIN Corporation,49,,
3,Feb 14 2023 12:08PM,257351,10,"Snap Medical Industries, LLC",1,,
4,Feb 14 2023 12:08PM,257349,10,ISDIN Corporation,3,,
5,Feb 14 2023 12:04PM,257348,20,"Exact-Rx, Inc.",6,,
6,Feb 14 2023 11:57AM,257347,10,"Citieffe, Inc.",1,,
7,Feb 14 2023 11:48AM,257345,20,Else Nutrition,1,,
8,Feb 14 2023 11:47AM,257346,10,MedStone Pharma LLC,7,,
9,Feb 14 2023 11:18AM,257343,10,"Methapharm, Inc.",21,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 14 2023 12:14PM,257353,10,Ivaoes Animal Health,1,,
1,Feb 14 2023 12:12PM,257352,10,"Nextsource Biotechnology, LLC",1,,
2,Feb 14 2023 12:12PM,257350,10,ISDIN Corporation,49,,
3,Feb 14 2023 12:08PM,257351,10,"Snap Medical Industries, LLC",1,,
4,Feb 14 2023 12:08PM,257349,10,ISDIN Corporation,3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 14 2023 12:14PM,257353,10,Ivaoes Animal Health,1.0,NaN,NaN
1,Feb 14 2023 12:12PM,257352,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
2,Feb 14 2023 12:12PM,257350,10,ISDIN Corporation,49.0,NaN,NaN
3,Feb 14 2023 12:08PM,257351,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
4,Feb 14 2023 12:08PM,257349,10,ISDIN Corporation,3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 14 2023 12:14PM,257353,10,Ivaoes Animal Health,1.0,0.0,0.0
1,Feb 14 2023 12:12PM,257352,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
2,Feb 14 2023 12:12PM,257350,10,ISDIN Corporation,49.0,0.0,0.0
3,Feb 14 2023 12:08PM,257351,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
4,Feb 14 2023 12:08PM,257349,10,ISDIN Corporation,3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6433174,258.0,8.0,0.0
12,514641,0.0,2.0,0.0
15,771829,10.0,52.0,0.0
16,257333,1.0,0.0,0.0
19,1543852,36.0,33.0,3.0
20,1029367,9.0,0.0,0.0
21,514596,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6433174,258.0,8.0,0.0
1,12,514641,0.0,2.0,0.0
2,15,771829,10.0,52.0,0.0
3,16,257333,1.0,0.0,0.0
4,19,1543852,36.0,33.0,3.0
5,20,1029367,9.0,0.0,0.0
6,21,514596,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,258.0,8.0,0.0
1,12,0.0,2.0,0.0
2,15,10.0,52.0,0.0
3,16,1.0,0.0,0.0
4,19,36.0,33.0,3.0
5,20,9.0,0.0,0.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,258.0
1,12,Released,0.0
2,15,Released,10.0
3,16,Released,1.0
4,19,Released,36.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,3.0,0.0,0.0
Executing,8.0,2.0,52.0,0.0,33.0,0.0,1.0
Released,258.0,0.0,10.0,1.0,36.0,9.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,Executing,8.0,2.0,52.0,0.0,33.0,0.0,1.0
2,Released,258.0,0.0,10.0,1.0,36.0,9.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,Executing,8.0,2.0,52.0,0.0,33.0,0.0,1.0
2,Released,258.0,0.0,10.0,1.0,36.0,9.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()